In [1]:
import pandas as pd
df = pd.read_csv("depression.csv")
df.drop(['Whichyear','Gender','Yourlocation','happywithlivingplace','donerecreationalactivitytoday','Age','Relationshipstatus', 'Understandingwithfamily','feelingpressureinyourstudy','supportsyouyouracademiclife','usedanysocialmedia','satisfiedwithmeal','feelingSick/healt issues'], axis=1, inplace=True)
df

,ExpressFeeling,Areyouhappyinancialy,succeededInEducationalinstitution,satisfiedwithacademicresult,haveinferioritycomplex,sleepAtNight,feelingrightnow
0,100,Yes,5,Yes,No,8,Very good
1,51,No,3,Yes,Maybe,8,Normal
2,87,Yes,4,Yes,No,7,Good
3,37,Yes,1,Yes,Maybe,5,Bad
4,56,No,4,No,No,7,Normal
...,...,...,...,...,...,...,...
226,96,Yes,4,No,No,10,Very good
227,24,Yes,2,No,No,7,Bad
228,91,Yes,5,Yes,No,8,Very good
229,86,Yes,3,No,No,9,Good


In [2]:
training_data = df.values[:,0:]
training_data


array([[100, 'Yes', 5, ..., 'No', 8, 'Very good'],
       [51, 'No', 3, ..., 'Maybe', 8, 'Normal'],
       [87, 'Yes', 4, ..., 'No', 7, 'Good'],
       ...,
       [91, 'Yes', 5, ..., 'No', 8, 'Very good'],
       [86, 'Yes', 3, ..., 'No', 9, 'Good'],
       [61, 'Yes', 3, ..., 'No', 6, 'Normal']], dtype=object)

In [3]:
header = ["ExpressFeeling","Areyouhappyinancialy","succeededInEducationalinstitution","satisfiedwithacademicresult","haveinferioritycomplex","sleepAtNight","feelingrightnow"]

In [4]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [5]:
unique_vals(training_data, 0)

{1,
 2,
 3,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 29,
 31,
 33,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 83,
 84,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100}

In [6]:
def class_counts(rows):
    counts = {}  
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
class_counts(training_data)

{'Very good': 48, 'Normal': 54, 'Good': 45, 'Bad': 37, 'Very bad': 47}

In [8]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [9]:
class Question:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [10]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [11]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [12]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [13]:
current_uncertainty = gini(training_data)
current_uncertainty

0.7971739660051348

In [14]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features): 
        values = set([row[col] for row in rows])  
        for val in values:  
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [15]:
best_gain, best_question = find_best_split(training_data)
best_question

Is ExpressFeeling >= 90?

In [16]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [17]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [18]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [19]:
def print_tree(node, spacing=""):
    
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [20]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is ExpressFeeling >= 90?
--> True:
  Predict {'Very good': 48}
--> False:
  Is ExpressFeeling >= 20?
  --> True:
    Is ExpressFeeling >= 71?
    --> True:
      Predict {'Good': 45}
    --> False:
      Is ExpressFeeling >= 50?
      --> True:
        Predict {'Normal': 54}
      --> False:
        Predict {'Bad': 37}
  --> False:
    Predict {'Very bad': 47}
